TITLE AND STUFF

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

INTRO
- CONTEXT
- DATA

PROBLEM DEF


IMPORT DATA

In [12]:
raw_df = pd.read_csv("Islander_data.csv")

CLEAN DATA

In [15]:
na_rows = raw_df.isna().sum().sum()  # Total count of NA values
print("Number of NA rows:", na_rows)

df = raw_df.drop(columns=['first_name', 'last_name'])

print(df.head())

Number of NA rows: 0
   age Happy_Sad_group  Dosage Drug  Mem_Score_Before  Mem_Score_After  Diff
0   25               H       1    A              63.5             61.2  -2.3
1   52               S       1    A              41.6             40.7  -0.9
2   29               H       1    A              59.7             55.1  -4.6
3   50               S       1    A              51.7             51.2  -0.5
4   52               H       1    A              47.0             47.1   0.1


EXPLORATORY DATA ANALYSIS

- MEM RECALL LENGTH VS DRUG TYPE (ANISH)

- DIFFERENT DOSAGE OF DRUG A AND MEAN DIFF FOR LENGTH (MATT)

- HIST OF AGE (KHUSHI)

- AVG DIFF OF MEM VS AGE (KHUSHI)

- MEM PERFORM BEFORE DRUG VS AGE (ANISH)

- AVG DIFF FOR EACH PRIMING BLOCK (MATT)

- DIFF IN MEM PERFORMANCE FOR EACH PRIMING BLOCK IN DRUG A (KHUSHI)

CONCLUSION